**MINI PROJECT 4 - Airport and Airlines**

**PART A**

In [1]:
import pandas as pd
from tabulate import tabulate
import traceback
import numpy as np
import matplotlib.pyplot as plt
from requests import get
import requests
import os

In [6]:
db1 = pd.read_csv('/content/drive/MyDrive/airports.dat.txt', header=None, names=["Airport_ID","Name","City","Country","IATA","ICAO","Latitude","Longitude","Altitude","Timezone","DST","Tz database timezone","Type","Source"])
db1.head(5)

,Airport_ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz database timezone,Type,Source
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports


**1) Top 20 highest altitude airports in the world. Choose only “type” with “airport”**


In [7]:
db1_airport = db1[db1['Type'] == 'airport']
db1_airport_sorted = db1_airport.sort_values('Altitude', ascending=False)
top20_airports = db1_airport_sorted.head(20)
top20_airports = top20_airports[['Airport_ID', 'Name', 'City', 'Country', 'Altitude']]
top20_airports['rank'] = range(1, len(top20_airports) + 1)
print(tabulate(top20_airports, headers='keys', tablefmt='psql'))

+------+--------------+----------------------------------------+-------------+-----------+------------+--------+
|      |   Airport_ID | Name                                   | City        | Country   |   Altitude |   rank |
|------+--------------+----------------------------------------+-------------+-----------+------------+--------|
| 6542 |         9310 | Daocheng Yading Airport                | Daocheng    | China     |      14472 |      1 |
| 5028 |         6396 | Qamdo Bangda Airport                   | Bangda      | China     |      14219 |      2 |
| 6393 |         8921 | Kangding Airport                       | Kangding    | China     |      14042 |      3 |
| 5872 |         7932 | Ngari Gunsa Airport                    | Shiquanhe   | China     |      14022 |      4 |
| 2625 |         2762 | El Alto International Airport          | La Paz      | Bolivia   |      13355 |      5 |
| 2627 |         2764 | Capitan Nicolas Rojas Airport          | Potosi      | Bolivia   |      

**2) Top 20 countries that have the highest total of airports.**

In [8]:
airport_counts = db1.groupby('Country')['Airport_ID'].count().sort_values(ascending=False).reset_index()
airport_counts = airport_counts.rename(columns={'Airport_ID': 'total_airports'})
airport_counts['rank'] = range(1, len(airport_counts) + 1)
top20_countries = airport_counts.head(20)
print(tabulate(top20_countries, headers='keys', tablefmt='psql'))

+----+----------------+------------------+--------+
|    | Country        |   total_airports |   rank |
|----+----------------+------------------+--------|
|  0 | United States  |             1512 |      1 |
|  1 | Canada         |              430 |      2 |
|  2 | Australia      |              334 |      3 |
|  3 | Russia         |              264 |      4 |
|  4 | Brazil         |              264 |      5 |
|  5 | Germany        |              249 |      6 |
|  6 | China          |              241 |      7 |
|  7 | France         |              217 |      8 |
|  8 | United Kingdom |              167 |      9 |
|  9 | India          |              148 |     10 |
| 10 | Indonesia      |              145 |     11 |
| 11 | Japan          |              123 |     12 |
| 12 | South Africa   |               99 |     13 |
| 13 | Argentina      |               96 |     14 |
| 14 | Mexico         |               84 |     15 |
| 15 | Italy          |               83 |     16 |
| 16 | Iran 

**3) Top 20 cities that have the highest total # of airports. (Two cities from different countries can have the same name, and you have to make sure the numbers are not messed up due to that!)**

In [11]:
city_counts = db1.groupby(['City','Country'])['Airport_ID'].count().sort_values(ascending=False).reset_index()
city_counts = city_counts.rename(columns={'Airport_ID': 'total_airports'})
city_counts['rank'] = range(1, len(city_counts) + 1)
top20_cities = city_counts.head(20)
print(tabulate(top20_cities, headers='keys', tablefmt='psql'))

+----+--------------+----------------+------------------+--------+
|    | City         | Country        |   total_airports |   rank |
|----+--------------+----------------+------------------+--------|
|  0 | Columbus     | United States  |                8 |      1 |
|  1 | Houston      | United States  |                6 |      2 |
|  2 | London       | United Kingdom |                6 |      3 |
|  3 | New York     | United States  |                6 |      4 |
|  4 | Moscow       | Russia         |                6 |      5 |
|  5 | Jacksonville | United States  |                6 |      6 |
|  6 | Izmir        | Turkey         |                5 |      7 |
|  7 | Vancouver    | Canada         |                5 |      8 |
|  8 | Greenville   | United States  |                5 |      9 |
|  9 | Atlanta      | United States  |                5 |     10 |
| 10 | Burlington   | United States  |                4 |     11 |
| 11 | Johannesburg | South Africa   |                4 |     

**4) Pair of cities that have the same name, but from two different countries.**

In [22]:
city_pairs = db1.merge(db1, on='City', suffixes=('_a', '_b'))
city_pairs = city_pairs[city_pairs['Country_a'] != city_pairs['Country_b']]
city_pairs = city_pairs[['City', 'Country_a', 'Country_b']].drop_duplicates()
city_pairs = city_pairs.dropna()
print(tabulate(city_pairs, headers='keys', tablefmt='psql'))

+-------+----------------+---------------------------+---------------------------+
|       | City           | Country_a                 | Country_b                 |
|-------+----------------+---------------------------+---------------------------|
|    47 | Princeton      | Canada                    | United States             |
|    59 | Eureka         | Canada                    | United States             |
|    68 | Kingston       | Canada                    | Jamaica                   |
|    73 | Geraldton      | Canada                    | Australia                 |
|    80 | Hamilton       | Canada                    | New Zealand               |
|    81 | Hamilton       | Canada                    | Australia                 |
|    82 | Hamilton       | Canada                    | United States             |
|    94 | Stephenville   | Canada                    | United States             |
|    98 | Waterloo       | Canada                    | United States             |
|   